# Imports

In [ ]:
# !pip install --upgrade transformers==4.49.0
# !pip install optuna==2.10.0
# !pip install numpy==1.26.4 gensim==4.3.2
# !pip install scipy==1.12.0
# !pip install --upgrade pandas==2.2.2
# !pip install h3
# !pip install mlflow
# !pip install 'protobuf<4'
# !pip install selenium
# !pip install natasha
# !pip install pymystem3
# !pip install symspellpy
# # !pip install nvidia-ml-py3
!rm -rf /content/Price-prediction-with-textual-data # для переустановки репозитория необходимо его сначала сбросить

In [ ]:
!git clone https://github.com/anna-k-00/Price-prediction-with-textual-data.git

In [ ]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Шаг 1: Проверка и настройка окружения
import os
import sys
import importlib

# Шаг 2: Клонирование/обновление репозитория
repo_url = 'https://github.com/anna-k-00/Price-prediction-with-textual-data.git'
repo_dir = 'Price-prediction-with-textual-data'

if not os.path.exists(repo_dir):
    !git clone {repo_url}
else:
    !cd {repo_dir} && git pull

# Шаг 3: Добавляем все нужные пути в sys.path
paths_to_add = [
    f'/content/{repo_dir}',                     # Для файлов в корне (parser_avito.py)
    f'/content/{repo_dir}/main_methods',        # Основные модули
    f'/content/{repo_dir}/embeddings_generation', # Генерация эмбеддингов
    f'/content/{repo_dir}/preprocessors'        # Препроцессоры
]

for path in paths_to_add:
    if os.path.exists(path) and path not in sys.path:
        sys.path.insert(0, path)
        print(f'Добавлен путь: {path}')

# Шаг 4: Собираем список всех модулей для импорта
all_modules = [
    # Основные модули
    'resource_monitor', 'ann', 'predict', 'test_pipeline',

    # Модули из embeddings_generation
    'embeddings_generation.rubert_transformer',
    'embeddings_generation.tfidf_transformer',
    'embeddings_generation.w2v_transformer',
    'embeddings_generation.gate',

    # Модули из preprocessors
    'preprocessors.preprocessor_params_hex',
    'preprocessors.preprocessor_text',

    # Отдельные файлы в корне
    'parser_avito'
]

# Шаг 5: Импортируем все модули
imported_modules = {}
failed_modules = {}

for module_name in all_modules:
    try:
        module = importlib.import_module(module_name)
        imported_modules[module_name] = module
        print(f'✅ {module_name} успешно импортирован')
    except Exception as e:
        failed_modules[module_name] = str(e)
        print(f'❌ Ошибка импорта {module_name}: {str(e)[:200]}')  # Обрезаем длинные сообщения

In [ ]:
import logging
import os
import time
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import optuna
from optuna.samplers import TPESampler
from tqdm import tqdm
import mlflow
import mlflow.sklearn
import warnings
from transformers import AutoModel, AutoTokenizer, AdamW
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from sklearn.svm import LinearSVR
from sklearn.base import BaseEstimator, TransformerMixin
import random
from datetime import datetime
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.sparse import issparse
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.base import BaseEstimator, TransformerMixin
import torch
import torch.nn as nn
import os
import joblib
import transformers
import json
import joblib
from datetime import datetime
from sklearn.decomposition import PCA
import time
import sys
import platform
import psutil
import threading

try:
    import pynvml
    pynvml.nvmlInit()
    HAS_NVML = True
except Exception:
    HAS_NVML = False

from ann import ANNRegressor
from resource_monitor import ResourceMonitor
from test_pipeline import PricePredictionExperiment
from preprocessor_params_hex import DataProcessingPipeline

from tfidf_transformer import TfidfTransformer
from w2v_transformer import Word2VecTransformer
from rubert_transformer import RuBertTiny2Embedder

# Prediction

### Mount at shared ML flow

самый простой способ получить доступ к нашему архиву ml flow - по ссылке 'https://drive.google.com/drive/folders/195Ie1O3SPhwsoMSiLTUydNI-9Tf_QnU7'

через интерфейс Google Drive:

- Перейдите по нашей ссылке будучи авторизованными (доступы уже открыты)
- Откройте Google Drive
- Найдите папку в разделе "Доступно мне" (Shared with me)
- Правой кнопкой мыши → "Добавить ярлык на Диск" (Add shortcut to Drive)
- Выберите расположение в "Мой диск" → Нажмите "Добавить"

In [ ]:
# после добавление шортката можно просто напрямую подключаться к папке - все результаты наших экспериментов уже у вас на диске
mlflow.set_tracking_uri('file:///content/drive/My Drive/mlflow_data')

### Download model

класс PricePredictorMLflow работает с моделями, лежащими в ml flow (со всеми их вариантами, которые можно сохранить во всех вариантах эксперимента)
он может найти для вас нужную модель и все связанные архивы
- либо по run_id (быстро)
- либо по param_filters смотря на лучший r^2 или на последний run - смотря что выберете

поскольку мы знаем, какая модель лучшая и знаем ее run_id - заберем ее

In [ ]:
predictor_base = PricePredictorMLflow(
    experiment_name="price_prediction",
    param_filters={
        "feature_set": "categorical-only",
        "use_pca": "False",
        'use_gate': 'False',
        'manual_text_params':'False',
    },
    run_id = 'd5df1a361ca244bd8d58886736a195f9', # для поиска по run id указывать параметры не обязательно, это пример для наглядности - в режиме false модель будет искаться по параметрам
    use_run_id = True,
    model_class=PricePredictionExperiment
)

### predict

для проверки работы модели, обученной на полном датасете рекомендуется использовать майские или апрельские объявления
из-за их размера они доступны только в google drive, а не в github
можно реализовать доступ по ссылкам
- май: https://drive.google.com/file/d/1EHjHXbjrpI7ZDE3Jd3bEzCDwSRokbAnR/view?usp=share_link
- апрель: https://drive.google.com/file/d/1WL7QVVafqrb-UAh3pTjVt0UfjfKLl-fj/view?usp=share_link

In [ ]:
import pandas as pd
from google.colab import drive

# Функция для загрузки файла по ID
def download_file_from_drive(file_id, output_name):
    !gdown "https://drive.google.com/uc?id={file_id}" -O {output_name}
    print(f"Файл {output_name} успешно скачан")
    !ls -lh {output_name}

# ID файлов из Google Drive (извлекаем из ссылок)
may_file_id = '1EHjHXbjrpI7ZDE3Jd3bEzCDwSRokbAnR'
april_file_id = '1WL7QVVafqrb-UAh3pTjVt0UfjfKLl-fj'  # Обратите внимание - такой же ID как у мая?


# Скачиваем файлы
download_file_from_drive(may_file_id, 'may_data.csv')
download_file_from_drive(april_file_id, 'april_data.csv')

# Функция для загрузки данных в DataFrame
def load_data_to_df(file_path):
    try:
        df = pd.read_csv(file_path)
        print(f"\nДанные из {file_path} успешно загружены:")
        print(f"Строк: {len(df)}, колонок: {len(df.columns)}")
        return df
    except Exception as e:
        print(f"\nОшибка при чтении {file_path}: {str(e)}")
        print("Пробуем альтернативные параметры загрузки...")
        try:
            df = pd.read_csv(file_path, encoding='windows-1251', sep=';')
            return df
        except Exception as e2:
            print(f"Не удалось загрузить файл: {str(e2)}")
            return None

# Загружаем данные
may = load_data_to_df('may_data.csv')
april = load_data_to_df('april_data.csv')

# Если данные загружены успешно - сохраняем их
if may is not None:
    may_output_path = '/content/drive/MyDrive/may_processed.csv'
    may.set_index('id').to_csv(may_output_path)
    print(f"\nДанные за май сохранены в: {may_output_path}")

if april is not None:
    april_output_path = '/content/drive/MyDrive/april_processed.csv'
    april.set_index('id').to_csv(april_output_path)
    print(f"Данные за апрель сохранены в: {april_output_path}")


In [ ]:
# формируем предсказания за апрель
result_base_april = predictor_base.predict(april)

In [ ]:
# оцениваем точность
preds_base_april = result_base_april['predictions']
preds_base_april = preds_base_april[preds_base_april['predicted_price'].notnull()]
metrics_base_april = predictor_base.evaluate_metrics(preds_base_april[preds_base_april['predicted_price'].notnull()])


print(f"R²: {metrics_base_april['r2']:.4f}")
print(f"RMSE: {metrics_base_april['rmse']:.2f}")
print(f"SMAPE: {metrics_base_april['smape']:.2f}%")
print(f"MedAPE: {metrics_base_april['medape']:.2f}%")

In [ ]:
result_base_may = predictor_base.predict(may)

In [ ]:
preds_base_may = result_base_may['predictions']
preds_base_may = preds_base_may[preds_base_may['predicted_price'].notnull()]
metrics_base_may= predictor_base.evaluate_metrics(preds_base_may[preds_base_may['predicted_price'].notnull()])


print(f"R²: {metrics_base_may['r2']:.4f}")
print(f"RMSE: {metrics_base_may['rmse']:.2f}")
print(f"SMAPE: {metrics_base_may['smape']:.2f}%")
print(f"MedAPE: {metrics_base_may['medape']:.2f}%")

### Пример определения фрода

In [ ]:
analyzer = RealEstateAnalyzer(
    preds_base_may
)
fraudulent = analyzer.fraud()

In [ ]:
from IPython.display import HTML


print("\nОчень одозрительные объявления:")
display(HTML(fraudulent[['predicted_price','price','relative_error','link']].to_html(escape=False, render_links=True)))

In [ ]:
analyzer = RealEstateAnalyzer(
    preds_base_may
)
suspicious = analyzer.suspicious()

print("\nПодозрительные объявления:")
display(HTML(suspicious[['predicted_price','price','relative_error','link']].to_html(escape=False, render_links=True)))